In [ ]:
#importing all the libraries required
import torch
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt
import gradio as gr


In [ ]:
# Load Stable Diffusion 1.5 (lighter, works well in Colab)
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")


In [ ]:
# Disable NSFW checker (avoid black images)
def dummy_checker(images, **kwargs):
    return images, [False]

pipe.safety_checker = dummy_checker

In [ ]:
#this function acts as the working unit for image generation
def generate(prompt, steps=250, scale=7.5, height=512, width=512):
    image = pipe(
        prompt,
        height=height,
        width=width,
        num_inference_steps=steps,
        guidance_scale=scale
    ).images[0]
    return image

In [ ]:
#using css to design the presentation of the generator
css = r"""
body {
    background: url("https://images.pexels.com/photos/114979/pexels-photo-114979.jpeg?cs=srgb&dl=pexels-veeterzy-114979.jpg&fm=jpg") no-repeat center center fixed;
    background-size: cover;
    font-family: Arial, sans-serif;
    color: black;
    margin: 0;
    padding: 0;
    display: flex;
    flex-direction: column;
    align-items: center;
}
h1 {
    display: flex;
    align-items: center;
    justify-content: center;
    background-color: rgba(200,150,255);
    color: white;
    padding: 12px 24px;
    border-radius: 10px;
    font-size: 28px;
    font-weight: bold;
    margin: 20px auto;
    text-align: center;
    gap: 10px; /* spacing between icon and text */
}
h1 img.title-icon {
    height: 28px;  /* adjust as needed */
    width: 28px;
    object-fit: contain;
}
#app {
    width: 80%;
    max-width: 1000px;
    margin: auto;
    text-align: center;
}
#generated-image {
    background: white;
    width: 100%;
    min-height: 400px;
    border-radius: 8px;
    display: flex;
    align-items: center;
    justify-content: center;
    margin-top: 15px;
}
input[type="range"] {
    -webkit-appearance: none;
    width: 100%;
    height: 6px;
    border-radius: 5px;
    background: #ddd; /* unfilled part */
    outline: none;
    border: none;
    padding: 0; /* removes blur/padding */
}
input[type="range"]::-webkit-slider-runnable-track {
    height: 6px;
    border-radius: 5px;
    background: #c896ff; /* filled track purple */
}

input[type="range"]::-webkit-slider-thumb {
    -webkit-appearance: none;
    appearance: none;
    width: 16px;
    height: 16px;
    border-radius: 50%;
    background: #c896ff; /* thumb purple */
    cursor: pointer;
    margin-top: -5px;
    border: none;
}
input[type="range"]::-moz-range-track {
    height: 6px;
    border-radius: 5px;
    background: #ddd; /* unfilled part */
}

input[type="range"]::-moz-range-progress {
    height: 6px;
    border-radius: 5px;
    background: #c896ff; /* filled purple */
}

input[type="range"]::-moz-range-thumb {
    width: 16px;
    height: 16px;
    border-radius: 50%;
    background: #c896ff;
    cursor: pointer;
    border: none;
}


"""


with gr.Blocks(css=css) as demo:
    gr.HTML('<div class="title-container"><h1>Text-to-Image Generator</h1></div>')
    gr.Markdown("Generate images interactively!")

    with gr.Row():
        prompt = gr.Textbox(label="Prompt", placeholder="Type your image prompt...")

    with gr.Row():
        steps = gr.Slider(1, 250, value=25, step=1, label="Inference Steps")
        scale = gr.Slider(1, 15, value=7.5, step=0.5, label="Guidance Scale")

    with gr.Row():
        height = gr.Slider(128, 1024, value=512, step=64, label="Height")
        width = gr.Slider(128, 1024, value=512, step=64, label="Width")

    output = gr.Image(type="pil", label="Generated Image")

    run_btn = gr.Button("Generate")

    run_btn.click(fn=generate, inputs=[prompt, steps, scale, height, width], outputs=output)

demo.launch(share=True)
